# <center> <font color = "green">A processing and analytics system for microscopy data workflows: the Pycroscopy ecosystem of packages
<img src = "https://github.com/pycroscopy/pycroscopy/blob/main/docs/px_logo_new.png?raw=true"> </img>
    
<center><font size = 3> R. Vasudevan<sup>1</sup>, S. M. Valleti,<sup>2</sup> M. Ziatdinov,<sup>1,3</sup> G. Duscher,<sup>4,</sup> and S. Somnath<sup>5,6</sup> </font></center>

<sup>1</sup>Center for Nanophase Materials Sciences, Oak Ridge National Laboratory \
<sup>2</sup>Bredesen Center for Interdisciplinary Research, University of Tennessee, Knoxville \
<sup>3</sup>Computational Sciences and Engineering Division, Oak Ridge National Laboratory \
<sup>4</sup>Department of Materials Science and Engineering, University of Tennessee, Knoxville \
<sup>5</sup>National Center for Computational Sciences, Oak Ridge National Laboratory \
<sup>6</sup>Current affiliation: McKinsey Consulting Group
    
This notebook accompanies our paper on the Pycroscopy Ecosystem of packages.

We will go through several examples of use cases. More information can be found <a href = "https://pycroscopy.github.io/pycroscopy/ecosystem.html">here</a>

## <left> Visualization, Spectral processing and Matrix Factorization with Pycroscopy
    
In this notebook we will go thorugh basics of loading files, visualizing them, fitting spectra to functions, and then more advanced image manipulation and matrix factorization
  

In [ ]:
#Load required packages
!pip install pyNSID sidpy SciFiReaders nanonispy gwyfile pycroscopy wget ipympl

In [ ]:
#download the required data
import wget
wget.download(url='https://github.com/pycroscopy/pycroscopy/blob/main/sample_data/bfo_iv_final.hf5?raw=true', 
              out = 'bfo_iv_final.hf5');
wget.download(url='https://github.com/pycroscopy/pycroscopy/blob/main/sample_data/bto_atomic.dm3?raw=true', 
              out = 'bto_atomic.dm3');
wget.download(url='https://github.com/pycroscopy/pycroscopy/blob/main/sample_data/re48.tiff?raw=true', 
              out = 're48.tiff');

In [ ]:
colab = True #Set to True if in Google Colab, else set to False
if colab:
  %matplotlib widget
  from google.colab import output
  output.enable_custom_widget_manager()
else:
  %matplotlib notebook

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np

import pyNSID
import sidpy as sid
import SciFiReaders as sr
from sidpy.proc.fitter import SidFitter

# Load and Visualize Spectral dataset

Here we load a dataset of I-V curves captured by conductive atomic force microscopy, on a BiFeO3 sample.
The data has been transformed so that we plot not the log of the current density J, as a function of the square root of the electric field. I-V curves plotted in this fashion will be linear if they follow Schottky emission cinduction laws. The dataset was originally presented in the paper <a href = "https://www.nature.com/articles/s41467-017-01334-5"> Vasudevan et al. Nat Commun. 8, 1318 (2017) </a>

Here we will load the file, which was saved as a pyNSID HDF5 file, using SciFiReaders, which reads it into a <i>sidpy.Dataset</i> object

Then we will use the .plot() method of the <i>sidpy.Dataset</i> object for interactive visualization.


In [ ]:
data_path = r'bfo_iv_final.hf5'
dr = sr.NSIDReader(data_path)
dataset_sid = dr.read()[0]
dataset_sid.plot();


# Functional Fitting

Much of the data appears linear. We can define any function of our choice and use sidpy's <i>SidFitter</i> class to make it easy to perform the fit on all of the spectra. We simply define the fit function, instantiate the class, and then call the do_fit() method.

Advantages of using the fitter class include:
- Innate scalability: we leverage the parallelism of Dask, so that the computations are performed in parallel
- Superior priors: We can use a k-means cluster approach to improve priors for the function fits, as described in <a href = "https://iopscience.iop.org/article/10.1088/2632-2153/abfbba/meta">Creange et al.</a>
- Dimension handling: There is intelligent folding and unfolding within the class to handle simple cases. For more complex situations (e.g., when there are multiple spectral dimensions, but only one is used for the fitting), these can be specified too.


In [ ]:
#Define the function we want each spectrum to

def one_lin_func(xvec, *coeff):
    a1,a2 = coeff
    return a1*xvec + a2

#Instantiate the SidFitter class
fitter = SidFitter(dataset_sid, one_lin_func,num_workers=4,
                           threads=2, return_cov=False, return_fit=True, return_std=False,
                           km_guess=True,num_fit_parms = 2)

In [ ]:
fit_parameters, fitted_dataset = fitter.do_fit() #With one line we can fit all the spectra

## Visualize Fit Results

We can visualize the parameter maps from the fititng, or visualize the fitted data curves

### Visualize coeffient maps

The functional fits provided us with coefficient maps for the fit parameters.

We can simply call the .plot() method again to visualize them.

In [ ]:
fit_parameters.plot();

### Visualize fitted spectra
Each spectrum is associated with a specific fit. 

We can visualize them again easily
through calling the <i>visualize_fit_results()</i> method of the SidFitter class.

In [ ]:
fitter.visualize_fit_results() # We can visualize the results of the fit interactively too...

# Save the results

We can easily save back to the original HDF5 file. 
This will help us when it comes to e.g., publications -> provide all the necessary data.



In [ ]:
import h5py
new_h5_filename = r'new_exp_file.hf5'
hf = h5py.File(new_h5_filename, 'a') #Create a new file

In [ ]:
#We can save the fitted results with pyNSID
hf_grp = hf.create_group('Measurement_000/Channel_000') #Create a group

#Let's save the raw data first
pyNSID.hdf_io.write_nsid_dataset(dataset_sid, hf_grp, main_data_name="IV_BFO_Raw")

In [ ]:
fitted_dataset.metadata

In [ ]:
sid.hdf_utils.print_tree(hf)

In [ ]:
hf_results_grp = hf['Measurement_000/Channel_000/IV_BFO_Raw'] #Let's put the results here.

pyNSID.hdf_io.write_nsid_dataset(fit_parameters, hf_results_grp, main_data_name="IV_BFO_Fit_Parameters")
pyNSID.hdf_io.write_nsid_dataset(fitted_dataset, hf_results_grp, main_data_name="IV_BFO_Fitted_Spectra")

sid.hdf_utils.print_tree(hf) #print result

hf.close()

# Second example - Image analysis

The pycroscopy package has some tools for generic image analysis, as well as wrappers around common machine learning methods. These include matrix and tensor factorization techniques. Let us explore one example.

First we will import a microscopy image, and then we will perform image windowing

We will then use matrix factorization to analyze the spatial distribution of different phases

This is explained in <a href = "https://pubs.acs.org/doi/full/10.1021/acs.nanolett.6b02130">this article</a>.


In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import pyNSID

import sidpy as sid
import SciFiReaders as sr
from sidpy.proc.fitter import SidFitter
import warnings
warnings.filterwarnings('ignore')

In [ ]:
dm3_file = r'bto_atomic.dm3'
png_file = r're48.tiff'

dm3_reader = sr.DM3Reader(dm3_file)
img_reader = sr.ImageReader(png_file)

data = dm3_reader.read()[0]

In [ ]:
data.plot();

In [ ]:
#Let's crop the image and try again
# Make a sidpy dataset

x_dim = data.dim_0.values
y_dim = data.dim_1.values

data_cropped = data[:][200:-200,10:550]

data_set = sid.Dataset.from_array(data_cropped, title='BTO_STEM')

# Set the data type
data_set.data_type = sid.DataType.IMAGE

# Add quantity and units
data_set.units = 'counts'
data_set.quantity = 'Intensity'

y_dim = y_dim[10:550]
x_dim = x_dim[200:-200]

# Add dimension info
data_set.set_dimension(0, sid.Dimension(x_dim,
                                        name='x',
                                        units='nm', quantity='x',
                                        dimension_type='spatial'))
data_set.set_dimension(1, sid.Dimension(y_dim,
                                        name='y',
                                        units='nm', quantity='y',
                                        dimension_type='spatial'))


In [ ]:
data_set.plot();

In [ ]:
from pycroscopy.image import ImageWindowing

parms_dict = {}
parms_dict['window_step_x'] = 8
parms_dict['window_step_y'] = 8
parms_dict['window_size_x'] = 128
parms_dict['window_size_y'] = 128
parms_dict['mode'] = 'fft'
parms_dict['filter'] = 'hamming'
parms_dict['zoom_factor'] = 2
parms_dict['interpol_factor'] = 2
iw = ImageWindowing(parms_dict)
windows = iw.MakeWindows(data_set)
windows = np.abs(np.log(np.abs(windows)))



In [ ]:
windows.plot();

In [ ]:
from pycroscopy.learn.ml.matrix_factor import MatrixFactor
mfactor = MatrixFactor(np.abs(windows), method = 'nmf',n_components = 2 )
output = mfactor.do_fit()

In [ ]:
abundances = output[0]
components = output[1]
abund = np.array(abundances)
comps = np.array(components)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, axes = plt.subplots(nrows=1, ncols=mfactor.ncomp, figsize = (10,3))
for ind, ax in enumerate(axes.flat):
    im1 = ax.imshow(comps[ind,:,:])
    ax.set_title('Component #' + str(ind))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im1, cax=cax, orientation='vertical')
    ax.axis('off')
fig.tight_layout()

fig, axes = plt.subplots(nrows=1, ncols=mfactor.ncomp, figsize = (10,3))
for ind, ax in enumerate(axes.flat):
    im1 = ax.imshow(abund[:,:,ind])
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im1, cax=cax, orientation='vertical')
    ax.axis('off')

fig.tight_layout()